##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFX パイプラインと TensorFlow Transform を使った特徴量エンジニアリング

***TFX パイプラインで入力データを変換し、モデルをトレーニングします。***

注：この例は、Jupyter スタイルのノートブックで今すぐ実行できます。セットアップは必要ありません。「Google Colab で実行」をクリックするだけです

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tft"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で実行</a></td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/penguin_transform.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colabで実行</a>
</td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/penguin_transform.ipynb">     <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tfx/tutorials/tfx/penguin_transform.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table></div>

ノートブックを使用したこのチュートリアルでは、生の入力データを取り込み、ML トレーニングに適するように前処理する TFX パイプラインを作成して実行します。このノートブックは、[TFX パイプラインと TensorFlow Data Validation を使ったデータ検証のチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfdv)を基盤としています。そのチュートリアルにまだ目を通していない場合は、このノートブックを進める前に読んでおくことをお勧めします。

特徴量エンジニアリングでは、データの予測の質を高めたり、次元を減らしたりすることができます。TFX を使用すると、変換コードを一度書けば、トレーニングとサービングの歪みを回避するために、変換がトレーニングとサービングで一貫して行われるというメリットがあります。

パイプラインに `Transform` コンポーネントを追加します。Transform コンポーネントは、[tf.transform](https://www.tensorflow.org/tfx/transform/get_started) ライブラリを使って実装されます。

TFX の様々な概念についての詳細は、[TFX パイプラインの理解](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)をご覧ください。

## セットアップ

まず、TFX Python パッケージをインストールし、モデルに使用するデータセットをダウンロードする必要があります。

### Pip のアップグレード

ローカルで実行する場合にシステムの Pip をアップグレードしないように、Colab で実行していることを確認してください。もちろん、ローカルシステムは個別にアップグレードできます。

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### TFX をインストールする


In [ ]:
!pip install -U tfx

### ランタイムを再起動しましたか？

Google Colab を使用している場合は、上記のセルを初めて実行した後に、「ランタイムを再起動」ボタンをクリックするか、「ランタイム」&gt;「ランタイムの再起動...」メニューを使って、ランタイムを再起動する必要があります。これは、Colab がパッケージを読み込むために必要な作業です。

TensorFlow と TFX のバージョンを確認します。

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### 変数をセットアップする

パイプラインを定義するために使用する変数がいくつかあります。これらの変数は、必要に応じてカスタマイズすることが可能です。デフォルトでは、パイプラインからのすべての出力は、現在のディレクトリに生成されます。

In [ ]:
import os

PIPELINE_NAME = "penguin-transform"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### サンプルデータを準備する

チュートリアルの TFX パイプラインで使用するサンプルデータセットをダウンロードします。使用するデータセットは [Palmer Penguins データセット](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)です。

ただし、前処理済みのデータセットを使用した前のチュートリアルとは異なり、**生**の Palmer Penguins データセットを使用します。


TFX ExampleGen コンポーネントはディレクトリから入力を読み取るため、ディレクトリを作成してデータセットをそれにコピーする必要があります。

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_path = 'https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins_size.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_path, _data_filepath)

生のデータがどのようなものか、さっと見てみましょう。

In [ ]:
!head {_data_filepath}

値が欠落したエントリがいくつかあります。これらは `NA` として表現されています。このチュートリアルでは、このようなエントリを削除することにします。

In [ ]:
!sed -i '/\bNA\b/d' {_data_filepath}
!head {_data_filepath}

ペンギンを説明する 7 つの特徴量があります。前のチュートリアルと同様に、同じ特徴量セット（'culmen_length_mm'、'culmen_depth_mm'、'flipper_length_mm'、'body_mass_g'）を使用して、ペンギンの 'species'（種）を予測します。

**唯一異なる点は、入力データが前処理されていないことです。** このチュートリアルでは、'island' や 'sex' などの他の特徴量は使用しないことに注意してください。

### スキーマファイルを準備する

[TFX パイプラインと TensorFlow Data Validation を使ったデータ検証のチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfdv)で説明されているとおり、データセットのスキーマファイルが必要です。このデータセットは前のチュートリアルとは異なるため、もう一度スキーマを生成する必要があります。このチュートリアルでは、このステップを省略して、準備しておいたスキーマファイルを使用します。


In [ ]:
import shutil

SCHEMA_PATH = 'schema'

_schema_uri = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/schema/raw/schema.pbtxt'
_schema_filename = 'schema.pbtxt'
_schema_filepath = os.path.join(SCHEMA_PATH, _schema_filename)

os.makedirs(SCHEMA_PATH, exist_ok=True)
urllib.request.urlretrieve(_schema_uri, _schema_filepath)

このスキーマファイルは、前のチュートリアルと同じパイプラインで作成されており、変更の手入れはありません。

## パイプラインを作成する

TFX パイプラインは Python API を使って定義されています。[データ検証チュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfdv)で作成したパイプラインに`Transform` コンポーネントを追加します。

Transform コンポーネントは、`ExampleGen` コンポーネントからの入力データと `SchemaGen` コンポーネントからのスキーマを必要とし、「変換グラフ」を生成します。出力は `Trainer` コンポーネントで使用されます。Transform はオプションとして、さらに「変換データ」も生成できます。これは、変換後にマテリアル化されたデータです。ただし、このチュートリアルでは、トレーニング中にデータを変換し、中間変換データのマテリアル化は行いません。

1 つ注意しておかなければならないのは、入力データがどのように変換されるかを説明する Python の `preprocessing_fn` 関数を定義する必要があることです。これは、モデル定義にユーザーコードを必要とする Trainer コンポーネントに似ています。


### 前処理とトレーニングのコードを記述する

2 つの Python 関数を定義する必要があります。1 つは Transform 用、もう 1 つは Trainer 用です。

#### preprocessing_fn

Transform コンポーネントは、`Trainer` コンポーネントと同様に、特定のモジュールファイルの `preprocessing_fn` 関数を見つけます。また、Transform コンポーネントの <a href="https://github.com/tensorflow/tfx/blob/142de6e887f26f4101ded7925f60d7d4fe9d42ed/tfx/components/transform/component.py#L113" data-md-type="link">`preprocessing_fn` パラメータを使用して、特定の関数を指定することもできます。</a>

この例では、2 種類の変換を行います。`culmen_length_mm` や `body_mass_g` などの連続した数値特徴量の場合は、[tft.scale_to_z_score](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_to_z_score) 関数を使って値を正規化します。ラベル特徴量の場合は、文字列ラベルを数値インデックス値に変換する必要があります。この変換には、[`tf.lookup.StaticHashTable`](https://www.tensorflow.org/api_docs/python/tf/lookup/StaticHashTable) を使用します。

変換したフィールドを見分けやすくするために、変換済みの特徴量名に `_xf` という接尾辞をアペンドします。

#### run_fn

モデル自体は前のチュートリアルとほとんど変わりませんが、今回は、Transform コンポーネントの変換グラフを使って入力データを変換します。

前のチュートリアルとのもう 1 つの重要な違いは、今回はモデルの計算グラフだけでなく、Transform コンポーネントで生成される前処理の変換グラフも含むサービング用のモデルをエクスポートすることです。着信リクエストを配信するために使用される別の関数を定義する必要があります。同じ `_apply_preprocessing` 関数がトレーニングデータとサービングリクエストの両方に使用されているのが分かります。


In [ ]:
_module_file = 'penguin_utils.py'

In [ ]:
%%writefile {_module_file}


from typing import List, Text
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_metadata.proto.v0 import schema_pb2
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

# Specify features that we will use.
_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10


# NEW: TFX Transform will call this function.
def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.

  Args:
    inputs: map from feature keys to raw not-yet-transformed features.

  Returns:
    Map from string feature key to transformed feature.
  """
  outputs = {}

  # Uses features defined in _FEATURE_KEYS only.
  for key in _FEATURE_KEYS:
    # tft.scale_to_z_score computes the mean and variance of the given feature
    # and scales the output based on the result.
    outputs[key] = tft.scale_to_z_score(inputs[key])

  # For the label column we provide the mapping from string to index.
  # We could instead use `tft.compute_and_apply_vocabulary()` in order to
  # compute the vocabulary dynamically and perform a lookup.
  # Since in this example there are only 3 possible values, we use a hard-coded
  # table for simplicity.
  table_keys = ['Adelie', 'Chinstrap', 'Gentoo']
  initializer = tf.lookup.KeyValueTensorInitializer(
      keys=table_keys,
      values=tf.cast(tf.range(len(table_keys)), tf.int64),
      key_dtype=tf.string,
      value_dtype=tf.int64)
  table = tf.lookup.StaticHashTable(initializer, default_value=-1)
  outputs[_LABEL_KEY] = table.lookup(inputs[_LABEL_KEY])

  return outputs


# NEW: This function will apply the same transform operation to training data
#      and serving requests.
def _apply_preprocessing(raw_features, tft_layer):
  transformed_features = tft_layer(raw_features)
  if _LABEL_KEY in raw_features:
    transformed_label = transformed_features.pop(_LABEL_KEY)
    return transformed_features, transformed_label
  else:
    return transformed_features, None


# NEW: This function will create a handler function which gets a serialized
#      tf.example, preprocess and run an inference with it.
def _get_serve_tf_examples_fn(model, tf_transform_output):
  # We must save the tft_layer to the model to ensure its assets are kept and
  # tracked.
  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_examples):
    # Expected input is a string which is serialized tf.Example format.
    feature_spec = tf_transform_output.raw_feature_spec()
    # Because input schema includes unnecessary fields like 'species' and
    # 'island', we filter feature_spec to include required keys only.
    required_feature_spec = {
        k: v for k, v in feature_spec.items() if k in _FEATURE_KEYS
    }
    parsed_features = tf.io.parse_example(serialized_tf_examples,
                                          required_feature_spec)

    # Preprocess parsed input with transform operation defined in
    # preprocessing_fn().
    transformed_features, _ = _apply_preprocessing(parsed_features,
                                                   model.tft_layer)
    # Run inference with ML model.
    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  dataset = data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(batch_size=batch_size),
      schema=tf_transform_output.raw_metadata.schema)

  transform_layer = tf_transform_output.transform_features_layer()
  def apply_transform(raw_features):
    return _apply_preprocessing(raw_features, transform_layer)

  return dataset.map(apply_transform).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [
      keras.layers.Input(shape=(1,), name=key)
      for key in _FEATURE_KEYS
  ]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # NEW: Save a computation graph including transform layer.
  signatures = {
      'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

これで、TFX パイプラインを構築するための準備ステップがすべて完了しました。

### パイプライン定義を記述する

TFX パイプラインを作成する関数を定義します。`Pipeline` オブジェクトは、TFX がサポートするパイプラインオーケストレーションシステムを使って実行できる TFX パイプラインです。


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     schema_path: str, module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Implements the penguin pipeline with TFX."""
  # Brings data into the pipeline or otherwise joins/converts training data.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # Import the schema.
  schema_importer = tfx.dsl.Importer(
      source_uri=schema_path,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer')

  # Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_importer.outputs['result'])

  # NEW: Transforms input data using preprocessing_fn in the 'module_file'.
  transform = tfx.components.Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_importer.outputs['result'],
      materialize=False,
      module_file=module_file)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],

      # NEW: Pass transform_graph to the trainer.
      transform_graph=transform.outputs['transform_graph'],

      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,
      statistics_gen,
      schema_importer,
      example_validator,

      transform,  # NEW: Transform component was added to the pipeline.

      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

## パイプラインの実行

前のチュートリアルのように、`LocalDagRunner` を使用します。

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      schema_path=SCHEMA_PATH,
      module_file=_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

パイプラインが正常に完了すると、「INFO:absl:Component Pusher is finished.」が表示されます。

Pusher コンポーネントは、トレーニングしたモデルを `SERVING_MODEL_DIR` にプッシュします。これは、前の手順で変数を変更していない場合は `serving_model/penguin-transform` ディレクトリとなります。Colab の左側のパネルにあるファイルブラウザで、または以下のコマンドを使うと、結果を確認できます。

In [ ]:
# List files in created model directory.
!find {SERVING_MODEL_DIR}

また、[`saved_model_cli` ツール](https://www.tensorflow.org/guide/saved_model#show_command)を使用して、生成されたモデルのシグネチャも確認できます。

In [ ]:
!saved_model_cli show --dir {SERVING_MODEL_DIR}/$(ls -1 {SERVING_MODEL_DIR} | sort -nr | head -1) --tag_set serve --signature_def serving_default

独自の `serve_tf_examples_fn` 関数を使って `serving_default` を定義しているため、シグネチャには、1 つの文字列を取ることが示されています。この文字列は tf.Examples のシリアル化された文字列で、前に定義したとおり、[tf.io.parse_example()](https://www.tensorflow.org/api_docs/python/tf/io/parse_example) 関数で解析されます（tf.Examples の詳細は、[こちら](https://www.tensorflow.org/tutorials/load_data/tfrecord) をご覧ください）。

エクスポートされたモデルを読み込み、いくつかの例を使って推論しみましょう。

In [ ]:
# Find a model with the latest timestamp.
model_dirs = (item for item in os.scandir(SERVING_MODEL_DIR) if item.is_dir())
model_path = max(model_dirs, key=lambda i: int(i.name)).path

loaded_model = tf.keras.models.load_model(model_path)
inference_fn = loaded_model.signatures['serving_default']

In [ ]:
# Prepare an example and run inference.
features = {
  'culmen_length_mm': tf.train.Feature(float_list=tf.train.FloatList(value=[49.9])),
  'culmen_depth_mm': tf.train.Feature(float_list=tf.train.FloatList(value=[16.1])),
  'flipper_length_mm': tf.train.Feature(int64_list=tf.train.Int64List(value=[213])),
  'body_mass_g': tf.train.Feature(int64_list=tf.train.Int64List(value=[5400])),
}
example_proto = tf.train.Example(features=tf.train.Features(feature=features))
examples = example_proto.SerializeToString()

result = inference_fn(examples=tf.constant([examples]))
print(result['output_0'].numpy())

'Gentoo' 種に対応する 3 つ目の要素が、3 つの内最も大きいと期待されています。

## Next steps

Transform コンポーネントについての詳細は、[Transform コンポーネントガイド](https://www.tensorflow.org/tfx/guide/transform)をご覧ください。その他のリソースは、https://www.tensorflow.org/tfx/tutorials に記載されています。

TFX の様々な概念についての詳細は、[TFX パイプラインの理解](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)をご覧ください。
